# SI within and outside its limits

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score
import rolldecay.paper_writing as paper_writing



In [ ]:
import rolldecayestimators.measure as measure
from rolldecay.froude_scaling import froude_scale
import seaborn as sns


In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.95, 
                             exclude_table_name='rolldecay_exclude')

df_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=-np.inf, 
                             exclude_table_name='rolldecay_exclude')



In [ ]:
df_ikeda.describe()

In [ ]:
df_rolldecay.describe()

In [ ]:
df_ = pd.merge(left=df_rolldecay, right=df_ikeda, how='inner', left_index=True, right_index=True, suffixes=('','_ikeda'))
df_.describe()

In [ ]:
db = database.get_db()

In [ ]:
df_rolldecay['ship_speed']*=1.852/3.6
df_ikeda['ship_speed']*=1.852/3.6

description = pd.read_sql_table('description', con=db.engine, index_col='id')
description.loc['ship_speed','unit']='m/s'
description.loc['VDES','unit']='m/s'
description.loc['Disp'] = {'description':'Ship discplacement','unit':'m3'}

df_ikeda['Disp']=df_ikeda['Volume']
df_rolldecay['Disp']=df_rolldecay['Volume']

skip=['omega0']  # This one is already model scale...
df_ikeda = froude_scale(data=df_ikeda, description=description, skip=skip)
df_rolldecay = froude_scale(data=df_rolldecay, description=description, skip=skip)

In [ ]:
df_compare = measure.linearized_matrix(df_rolldecay=df_rolldecay, df_ikeda=df_ikeda)
#mask = df_compare['A0'] > 0.95
#df_compare=df_compare.loc[mask].copy()

In [ ]:
df_compare.describe()

In [ ]:
len(df_compare['id'].unique())

In [ ]:
df_compare.head()

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_e_hat', y='B_e_hat_ikeda', ax=ax, style='.')

xlim = ax.get_xlim()
ylim = ax.get_ylim()
lim = np.max([xlim[1],ylim[1]])
ax.set_xlim(0,lim)
ax.set_ylim(0,lim)
ax.plot([0,lim],[0,lim],'r-')
ax.set_aspect('equal', 'box')
ax.legend()
ax.grid(True)

In [ ]:
r2_score(y_true=df_compare['B_e_hat'], y_pred=df_compare['B_e_hat_ikeda'])

In [ ]:
df_compare['error'] = (df_compare['B_e_hat_ikeda'] - df_compare['B_e_hat']).abs()

## Now remove point outside the limits

In [ ]:
from rolldecayestimators.simplified_ikeda import limits_kawahara

In [ ]:
limits_kawahara=pd.Series(limits_kawahara)

In [ ]:
df_compare['g']=9.81
df_compare['draught']=(df_compare['TA'] + df_compare['TF'])/2
df_compare['CB'] = df_compare['Volume']/(df_compare['lpp']*df_compare['beam']*df_compare['draught'])
df_compare[r'B/d']=df_compare['beam']/df_compare['draught']
df_compare['OG']=df_compare['draught']-df_compare['kg']
df_compare[r'OG/d']=df_compare['OG']/df_compare['draught']
df_compare['CMID']=df_compare['A0']
df_compare[r'bBk/B']=df_compare['BKB']/df_compare['beam']
df_compare[r'lBk/LPP']=df_compare['BKL']/df_compare['lpp']
df_compare['OMEGA_hat']=lambdas.omega_hat(beam=df_compare['beam'], g=df_compare['g'], omega0=df_compare['omega0'])

In [ ]:
limit_keys = list(limits_kawahara.keys())
corr = df_compare[limit_keys].corr().abs()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, vmin=0, vmax=1, cmap='gist_yarg', annot=True, mask = mask)

In [ ]:
sns.pairplot(data=df_compare, x_vars=limit_keys, y_vars=limit_keys, markers='.', height=1.5)

In [ ]:
def compare_versus(x,y):
    fig,ax=plt.subplots()

    df_compare.plot(x=x,y=y, style='.', ax=ax)
    ax.set_ylabel(y)
    xlim=ax.get_xlim()
    ylim=ax.get_ylim()
    alpha=0.7
    
    y_=[np.max([ylim[0],limits_kawahara[y][0]]),
      np.min([ylim[1],limits_kawahara[y][1]]),
      ]
    ax.fill_between(x=xlim, y1=y_[0], y2=y_[1], color='g', alpha=alpha)
    
    x_=[np.max([xlim[0],limits_kawahara[x][0]]),
      np.min([xlim[1],limits_kawahara[x][1]]),
      ]
    ax.fill_between(x=x_, y1=ylim[0], y2=ylim[1], color='g', alpha=alpha);

    return ax

In [ ]:
compare_versus(x='CMID', y='B/d');

In [ ]:
compare_versus(x='B/d', y='bBk/B');

In [ ]:
compare_versus(x='CMID', y='bBk/B');

In [ ]:

g = sns.jointplot(x=r'B/d', y='error', data=df_compare,
                  kind="reg", truncate=False,
                  color="m", height=7)

g = sns.jointplot(x=r'CMID', y='error', data=df_compare,
                  kind="reg", truncate=False,
                  color="m", height=7)

g = sns.jointplot(x=r'OMEGA_hat', y='error', data=df_compare,
                  kind="reg", truncate=False,
                  color="m", height=7)

In [ ]:
corr = df_compare[limit_keys+['error']].corr().abs()

fig,ax=plt.subplots()
corr.plot(y='error', kind='bar', ax=ax);
ax.set_title('Absolut correlation coefficient for the error');

In [ ]:
fig,axes=plt.subplots(nrows=len(limits_kawahara))
fig.set_size_inches(15,12)

for (item, limits),ax in zip(limits_kawahara.items(),axes):
        
    df_compare.hist(item, ax=ax)
    ax.set_xlabel(item)
    ax.set_title('')
    
    ylims = ax.get_ylim()
    ax.fill_between(limits, [ylims[1],ylims[1]], y2=0, color='green', alpha=0.5, label='valid')
    
plt.tight_layout()
    

In [ ]:
def cut(df, limits, item):
    df=df.copy()
    mask = ((df[item] >= limits[0]) & (df[item] <= limits[1])) | (df[item]==0)
    df=df.loc[mask].copy()
    return df

def plot_select(df_compare, limits_used):
    
    df_compare_within = df_compare.copy()

    fig,axes=plt.subplots(nrows=len(limits_used))
    fig.set_size_inches(15,12)
    
    for (item, limits),ax in zip(limits_used.items(),axes):
        
        n_bins = 20
        bins=np.linspace(df_compare[item].min(), df_compare[item].max(),n_bins)
        df_compare.hist(item, ax=ax, bins=bins, color='gray')
        
        df_compare_within = cut(df=df_compare_within, limits=limits, item=item)
        df_compare_within.hist(item, ax=ax, bins=bins, color='blue')
        ax.set_xlabel(item)
        ax.set_title('')
        
        ylims = ax.get_ylim()
        ax.fill_between(limits, [ylims[1],ylims[1]], y2=0, color='green', alpha=0.5, label='valid')
        
    plt.tight_layout()
    
    fig,ax=plt.subplots()
    df_compare_within.plot(x='B_e_hat', y='B_e_hat_ikeda', ax=ax, style='.')
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    lim = np.max([xlim[1],ylim[1]])
    ax.set_xlim(0,lim)
    ax.set_ylim(0,lim)
    ax.plot([0,lim],[0,lim],'r-')
    ax.set_aspect('equal', 'box')
    ax.legend()
    ax.grid(True)
    
    r2 = r2_score(y_true=df_compare_within['B_e_hat'], y_pred=df_compare_within['B_e_hat_ikeda'])
    print('r2:%f' % r2)
    
    return df_compare_within
        

In [ ]:
limits_kawahara

If we put **bBk/B** and **CMID** first, the other limits becomes redundant:

In [ ]:
keys_ordered=[
'bBk/B',     
'CMID',    
'CB',        
'B/d',       
'OG/d',      
'lBk/LPP',   
'OMEGA_hat',
]

limits_used = limits_kawahara[keys_ordered]
df_compare_within = plot_select(df_compare=df_compare, limits_used=limits_used)

In [ ]:
limits_used = limits_kawahara.copy()
limits_used.pop('CMID')
df_compare_within = plot_select(df_compare=df_compare, limits_used=limits_used)


In [ ]:
def select(df_compare, limits_used):
    
    df_compare_within = df_compare.copy()
    
    for item, limits in limits_used.items():
        
        df_compare_within = cut(df=df_compare_within, limits=limits, item=item)
        
    return df_compare_within

In [ ]:
limits_used = limits_kawahara.copy()
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
len(df_['id'].unique())

In [ ]:
limits_used = limits_kawahara.copy()
limits_used.pop(r'CMID')

df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = {}
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['B/d']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['B/d','bBk/B']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['B/d','bBk/B','CMID']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['CMID']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['CMID','bBk/B']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
len(df_['id'].unique())

In [ ]:
limits_used = limits_kawahara[['CMID','B/d']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

In [ ]:
limits_used = limits_kawahara[['CMID','bBk/B']]
df_ = select(df_compare=df_compare, limits_used=limits_used)
r2_score(y_true=df_['B_e_hat'], y_pred=df_['B_e_hat_ikeda'])

df_compare_within = plot_select(df_compare=df_compare, limits_used=limits_used)